### дополнительная предобработка текста

уже есть:
- Перевод всех букв в тексте в нижний или верхний регистры
- Удаление пунктуации
- Токенизация 

сделать:
- Удаление цифр (чисел) или замена на текстовый эквивалент 
- Удаление стоп слов
- Стемминг
- Лемматизация
- Векторизация

In [62]:
import pandas as pd

In [63]:
filename = 'data_hot.csv'
df = pd.read_csv(filename)

In [64]:
df.columns

Index(['Unnamed: 0', 'FileName', 'Title', 'Link', 'ArticleId', 'Date', 'Views',
       'Author', 'Tags', 'AmountComments', 'Rating', 'Text'],
      dtype='object')

In [54]:
# лемматизация

import pymorphy2
morph = pymorphy2.MorphAnalyzer()

words = ['грустно', 'зависимость', 'хорошему', 'приводит', 'альтернатив']
for word in words:
    p = morph.parse(word)[0]
    print(p.normal_form)

грустно
зависимость
хороший
приводить
альтернатива


In [57]:
# стемминг
# -*- coding: utf-8 -*-
# https://gist.github.com/Kein1945/9111512
import re

class Porter:
    PERFECTIVEGROUND =  re.compile(u"((ив|ивши|ившись|ыв|ывши|ывшись)|((?<=[ая])(в|вши|вшись)))$")
    REFLEXIVE = re.compile(u"(с[яь])$")
    ADJECTIVE = re.compile(u"(ее|ие|ые|ое|ими|ыми|ей|ий|ый|ой|ем|им|ым|ом|его|ого|ему|ому|их|ых|ую|юю|ая|яя|ою|ею)$")
    PARTICIPLE = re.compile(u"((ивш|ывш|ующ)|((?<=[ая])(ем|нн|вш|ющ|щ)))$")
    VERB = re.compile(u"((ила|ыла|ена|ейте|уйте|ите|или|ыли|ей|уй|ил|ыл|им|ым|ен|ило|ыло|ено|ят|ует|уют|ит|ыт|ены|ить|ыть|ишь|ую|ю)|((?<=[ая])(ла|на|ете|йте|ли|й|л|ем|н|ло|но|ет|ют|ны|ть|ешь|нно)))$")
    NOUN = re.compile(u"(а|ев|ов|ие|ье|е|иями|ями|ами|еи|ии|и|ией|ей|ой|ий|й|иям|ям|ием|ем|ам|ом|о|у|ах|иях|ях|ы|ь|ию|ью|ю|ия|ья|я)$")
    RVRE = re.compile(u"^(.*?[аеиоуыэюя])(.*)$")
    DERIVATIONAL = re.compile(u".*[^аеиоуыэюя]+[аеиоуыэюя].*ость?$")
    DER = re.compile(u"ость?$")
    SUPERLATIVE = re.compile(u"(ейше|ейш)$")
    I = re.compile(u"и$")
    P = re.compile(u"ь$")
    NN = re.compile(u"нн$")

    def stem(word):
        word = word.lower()
        word = word.replace(u'ё', u'е')
        m = re.match(Porter.RVRE, word)
        if m.groups():
            pre = m.group(1)
            rv = m.group(2)
            temp = Porter.PERFECTIVEGROUND.sub('', rv, 1)
            if temp == rv:
                rv = Porter.REFLEXIVE.sub('', rv, 1)
                temp = Porter.ADJECTIVE.sub('', rv, 1)
                if temp != rv:
                    rv = temp
                    rv = Porter.PARTICIPLE.sub('', rv, 1)
                else:
                    temp = Porter.VERB.sub('', rv, 1)
                    if temp == rv:
                        rv = Porter.NOUN.sub('', rv, 1)
                    else:
                        rv = temp
            else:
                rv = temp

            rv = Porter.I.sub('', rv, 1)

            if re.match(Porter.DERIVATIONAL, rv):
                rv = Porter.DER.sub('', rv, 1)

            temp = Porter.P.sub('', rv, 1)
            if temp == rv:
                rv = Porter.SUPERLATIVE.sub('', rv, 1)
                rv = Porter.NN.sub(u'н', rv, 1)
            else:
                rv = temp
            word = pre+rv
        return word
    stem=staticmethod(stem)

if __name__ == '__main__':
    print(Porter.stem(u'устойчивость'))

устойчив


In [17]:
from string import punctuation
# import gensim
import string
from nltk.tokenize import sent_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize

In [7]:
import nltk
nltk.download("stopwords")

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\138904\AppData\Roaming\nltk_data...
[nltk_data]   Unzipping corpora\stopwords.zip.


True

In [26]:
text = 'Давайте рассмотрим, пожалуй самый простой пример использования библиотеки NLTK токенизацию'.split(" ")
# print(word_tokenize(text))

stop_words = stopwords.words('russian')

print(stop_words)

['и', 'в', 'во', 'не', 'что', 'он', 'на', 'я', 'с', 'со', 'как', 'а', 'то', 'все', 'она', 'так', 'его', 'но', 'да', 'ты', 'к', 'у', 'же', 'вы', 'за', 'бы', 'по', 'только', 'ее', 'мне', 'было', 'вот', 'от', 'меня', 'еще', 'нет', 'о', 'из', 'ему', 'теперь', 'когда', 'даже', 'ну', 'вдруг', 'ли', 'если', 'уже', 'или', 'ни', 'быть', 'был', 'него', 'до', 'вас', 'нибудь', 'опять', 'уж', 'вам', 'ведь', 'там', 'потом', 'себя', 'ничего', 'ей', 'может', 'они', 'тут', 'где', 'есть', 'надо', 'ней', 'для', 'мы', 'тебя', 'их', 'чем', 'была', 'сам', 'чтоб', 'без', 'будто', 'чего', 'раз', 'тоже', 'себе', 'под', 'будет', 'ж', 'тогда', 'кто', 'этот', 'того', 'потому', 'этого', 'какой', 'совсем', 'ним', 'здесь', 'этом', 'один', 'почти', 'мой', 'тем', 'чтобы', 'нее', 'сейчас', 'были', 'куда', 'зачем', 'всех', 'никогда', 'можно', 'при', 'наконец', 'два', 'об', 'другой', 'хоть', 'после', 'над', 'больше', 'тот', 'через', 'эти', 'нас', 'про', 'всего', 'них', 'какая', 'много', 'разве', 'три', 'эту', 'моя', 'впр

In [27]:
tokens = [i for i in text if (i not in stop_words)]
print(tokens)

['Давайте', 'рассмотрим,', 'пожалуй', 'самый', 'простой', 'пример', 'использования', 'библиотеки', 'NLTK', 'токенизацию']


In [29]:
from nltk.stem.snowball import SnowballStemmer
stemmer = SnowballStemmer("russian") 

stem = [stemmer.stem(word) for word in tokens]
print(stem)

['дава', 'рассмотрим,', 'пожал', 'сам', 'прост', 'пример', 'использован', 'библиотек', 'NLTK', 'токенизац']


In [31]:
import nltk
nltk.download("stopwords")
#--------#

from nltk.corpus import stopwords
from pymystem3 import Mystem
from string import punctuation

#Create lemmatizer and stopwords list
mystem = Mystem() 
russian_stopwords = stopwords.words("russian")

#Preprocess function
def preprocess_text(text):
    tokens = mystem.lemmatize(text.lower())
    tokens = [token for token in tokens if token not in russian_stopwords\
              and token != " " \
              and token.strip() not in punctuation]
    
    text = " ".join(tokens)
    
    return text

#Examples    
preprocess_text("Ну что сказать, я вижу кто-то наступил на грабли, Ты разочаровал меня, ты был натравлен.")
#> 'сказать видеть кто-то наступать грабли разочаровывать натравлять'

preprocess_text("По асфальту мимо цемента, Избегая зевак под аплодисменты. Обитатели спальных аррондисманов")
#> 'асфальт мимо цемент избегать зевака аплодисменты обитатель спальный аррондисман'

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\138904\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
Installing mystem to C:\Users\138904/.local/bin\mystem.exe from http://download.cdn.yandex.net/mystem/mystem-3.1-win-64bit.zip


'асфальт мимо цемент избегать зевака аплодисменты обитатель спальный аррондисман'

In [35]:
lem_text = preprocess_text("По асфальту мимо цемента, Избегая зевак под аплодисменты. Обитатели спальных аррондисманов").split(" ")
print(lem_text)

['асфальт', 'мимо', 'цемент', 'избегать', 'зевака', 'аплодисменты', 'обитатель', 'спальный', 'аррондисман']


In [36]:
stem = [stemmer.stem(word) for word in lem_text]
print(stem)

['асфальт', 'мим', 'цемент', 'избега', 'зевак', 'аплодисмент', 'обитател', 'спальн', 'аррондисма']


In [50]:
text = "Как приготовить банановые оладьи:Для начала разомните банан вилкой, превратите его в однородную кашицу.\
Добавьте к банану яйцо. Перемешайте. Добавьте молоко и муку. Размешайте до однородного состояния."

from nltk.stem import PorterStemmer, WordNetLemmatizer
from nltk.corpus import wordnet

lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

print(stemmer.stem("Коронавирусный"))
print(lemmatizer.lemmatize("Коронавирусный", wordnet.VERB))

print(stemmer.stem("Коронный"))
print(lemmatizer.lemmatize("Коронный", wordnet.VERB))


коронавирусный
Коронавирусный
коронный
Коронный


In [51]:
preprocess_text("Коронный")

'коронный'

In [52]:
stemmer.stem("Коронный")

'коронный'

In [55]:
p = morph.parse("Коронный")[0]
print(p.normal_form)

коронный


In [58]:
Porter.stem(u'Коронный')

'корон'

In [60]:
print(lemmatizer.lemmatize("корон", wordnet.VERB))

корон


### Векторизация

In [67]:
documents = ["I like this movie, movie is funny.", 'I hate this movie.', 'This was awesome! I like it.', 'Nice one. I love it.']

In [68]:
from sklearn.feature_extraction.text import CountVectorizer
import pandas as pd

count_vectorizer = CountVectorizer()

bag_of_words = count_vectorizer.fit_transform(documents)

feature_names = count_vectorizer.get_feature_names()
pd.DataFrame(bag_of_words.toarray(), columns = feature_names)

,awesome,funny,hate,is,it,like,love,movie,nice,one,this,was
0,0,1,0,1,0,1,0,2,0,0,1,0
1,0,0,1,0,0,0,0,1,0,0,1,0
2,1,0,0,0,1,1,0,0,0,0,1,1
3,0,0,0,0,1,0,1,0,1,1,0,0


In [69]:
from sklearn.feature_extraction.text import TfidfVectorizer
import pandas as pd

tfidf_vectorizer = TfidfVectorizer()
values = tfidf_vectorizer.fit_transform(documents)

# Show the Model as a pandas DataFrame
feature_names = tfidf_vectorizer.get_feature_names()
pd.DataFrame(values.toarray(), columns = feature_names)

,awesome,funny,hate,is,it,like,love,movie,nice,one,this,was
0,0.000000,0.425807,0.000000,0.425807,0.000000,0.335711,0.000000,0.671422,0.000000,0.000000,0.271787,0.000000
1,0.000000,0.000000,0.702035,0.000000,0.000000,0.000000,0.000000,0.553492,0.000000,0.000000,0.448100,0.000000
2,0.523381,0.000000,0.000000,0.000000,0.412640,0.412640,0.000000,0.000000,0.000000,0.000000,0.334067,0.523381
3,0.000000,0.000000,0.000000,0.000000,0.414289,0.000000,0.525473,0.000000,0.525473,0.525473,0.000000,0.000000
